In [1]:
import pandas as pd
import math
import pickle
import numpy as np

from sklego.metrics import equal_opportunity_score, p_percent_score
from sklearn.metrics import accuracy_score
from fair_models import coefficient_of_variation

from moopt import monise
from fair_models import FairScalarization, EqualScalarization
from fair_models import calc_reweight
from sklearn.linear_model import LogisticRegression

import plotly.graph_objects as go

import sys
sys.path.append("./MMFP/")
from MMPF.MinimaxParetoFair.MMPF_trainer import SKLearn_Weighted_LLR, APSTAR

In [2]:
from sklego.linear_model import DemographicParityClassifier
from sklego.linear_model import EqualOpportunityClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
# Minimax - Marrom, LogReg - Amarelo, Reweight - Verde, ParDem - Roxo, EqOP - Laranja
# MOErro - Azul, MooAcep - Vermelho
hist_color = {'Minimax': '#a65628', 'LogReg':'#f781bf', 'Reweight':'#4daf4a', 
                'DemPar': '#984ea3', 'EqOp': '#ff7f00', 'MOOError': '#377eb8','MOOAcep': '#e41a1c'}
line_color = {'Minimax': '#c9997e', 'LogReg':'#f781bf', 'Reweight':'#94cf92', 
                'DemPar': '#c194c7', 'EqOp': '#ffb266', 'MOOError': '#87b1d4','MOOAcep': '#f18c8d'}

# Dataset

In [4]:
#dataset = 'compas'
#fair_feature = 'race'

# dataset = 'german'
# fair_feature = 'sex'

dataset = 'adult'
fair_feature = 'race'

In [5]:
X_train = pd.read_pickle("data/"+dataset+"_"+fair_feature+"/X_train.pickle")
with open("data/"+dataset+"_"+fair_feature+"/y_train.pickle", 'rb') as f:
    y_train = pickle.load(f)

X_val = pd.read_pickle("data/"+dataset+"_"+fair_feature+"/X_valid.pickle")
with open("data/"+dataset+"_"+fair_feature+"/y_valid.pickle", 'rb') as f:
    y_val = pickle.load(f)

X_test = pd.read_pickle("data/"+dataset+"_"+fair_feature+"/X_test.pickle")
with open("data/"+dataset+"_"+fair_feature+"/y_test.pickle", 'rb') as f:
    y_test = pickle.load(f)

In [6]:
other_solutions_list = {'Minimax': [], 'LogReg':[], 'Reweight':[], 'DemPar':[], 'EqOp':[]}

# Experimento 1 - Diversidade

No primeiro experimento, geramos diversos modelos diferentes com as estratégias selecionadas (propostas e comparadas) ao explorar parâmetros diferentes para cada estratégia. Avaliamos, visualmente através de gráficos de coordenadas paralelas, a diversidade que cada estratégia conseguiu gerar.

## Modelo 1) Fair Moo - Erro por grupo

In [7]:
moo_err = monise(weightedScalar=FairScalarization(X_train, y_train, fair_feature),
                 singleScalar=FairScalarization(X_train, y_train, fair_feature),
                  nodeTimeLimit=2, targetSize=150,
                  targetGap=0, nodeGap=0.05, norm=False)

moo_err.optimize()

2021-08-23 10:24:44,118 - moopt.monise - DEBUG: Finding 1th individual minima
2021-08-23 10:24:45,915 - moopt.monise - DEBUG: Finding 2th individual minima
2021-08-23 10:24:47,172 - moopt.monise - DEBUG: Finding 3th individual minima


Academic license - for non-commercial use only - expires 2021-10-17
Using license file /home/viguardieiro/gurobi.lic


2021-08-23 10:24:47,826 - moopt.monise - DEBUG: 4th solution - importance: 1.0
2021-08-23 10:24:48,225 - moopt.monise - DEBUG: 5th solution - importance: 1.0
2021-08-23 10:24:48,660 - moopt.monise - DEBUG: 6th solution - importance: 1.0
2021-08-23 10:24:48,969 - moopt.monise - DEBUG: 7th solution - importance: 1.0
2021-08-23 10:24:49,580 - moopt.monise - DEBUG: 8th solution - importance: 1.0
2021-08-23 10:24:50,023 - moopt.monise - DEBUG: 9th solution - importance: 0.7023875865404325
2021-08-23 10:24:50,681 - moopt.monise - DEBUG: 10th solution - importance: 0.32671411860660954
2021-08-23 10:24:52,088 - moopt.monise - DEBUG: 11th solution - importance: 0.12870498203730596
2021-08-23 10:24:52,550 - moopt.monise - DEBUG: 12th solution - importance: 0.10448248301470564
2021-08-23 10:24:53,192 - moopt.monise - DEBUG: 13th solution - importance: 0.1018767797346663
2021-08-23 10:24:53,555 - moopt.monise - DEBUG: 14th solution - importance: 0.08988015357276015
2021-08-23 10:24:54,211 - moopt.

In [8]:
m_aux = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}
sols = []

for solution in moo_err.solutionsList:
    sols.append(solution.x)
    m_aux["Acurácia"].append(solution.x.score(X_val, y_val))
    m_aux["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(solution.x,
                                                                                                            X_val, y_val))
    m_aux["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(solution.x,X_val))
    m_aux["Coeficiente de variação"].append(coefficient_of_variation(solution.x, X_val, y_val))

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  warnings.warn(
/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning: No samples with y_hat == 1 for race == 1, returning 0
  warnings.warn(
/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWar

In [9]:
m_metrics = pd.DataFrame(data=m_aux, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [10]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['MOOError'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = m_metrics['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = m_metrics['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = m_metrics['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = m_metrics['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

## Modelo 2) Equal - Aceitação por grupo

In [11]:
moo_prob = monise(weightedScalar=EqualScalarization(X_train, y_train, fair_feature),
             singleScalar=EqualScalarization(X_train, y_train, fair_feature),
              nodeTimeLimit=2, targetSize=150,
              targetGap=0, nodeGap=0.01, norm=False)

moo_prob.optimize()

2021-08-23 10:28:31,266 - moopt.monise - DEBUG: Finding 1th individual minima
2021-08-23 10:28:34,111 - moopt.monise - DEBUG: Finding 2th individual minima
2021-08-23 10:28:35,665 - moopt.monise - DEBUG: Finding 3th individual minima
2021-08-23 10:28:37,299 - moopt.monise - DEBUG: Finding 4th individual minima
2021-08-23 10:28:39,480 - moopt.monise - DEBUG: 5th solution - importance: 1.0
2021-08-23 10:28:41,379 - moopt.monise - DEBUG: 6th solution - importance: 1.0
2021-08-23 10:28:42,745 - moopt.monise - DEBUG: 7th solution - importance: 1.0
2021-08-23 10:28:44,475 - moopt.monise - DEBUG: 8th solution - importance: 1.0
2021-08-23 10:28:46,050 - moopt.monise - DEBUG: 9th solution - importance: 1.0
2021-08-23 10:28:47,355 - moopt.monise - DEBUG: 10th solution - importance: 0.7664892179397275
2021-08-23 10:28:49,014 - moopt.monise - DEBUG: 11th solution - importance: 0.4365654326287781
2021-08-23 10:28:50,413 - moopt.monise - DEBUG: 12th solution - importance: 0.3520369516714202
2021-08-

In [12]:
sols = []

m_aux2 = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}
for solution in moo_prob.solutionsList:
    sols.append(solution.x)
    m_aux2["Acurácia"].append(solution.x.score(X_test, y_test))
    m_aux2["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(solution.x,
                                                                                                            X_test, y_test))
    m_aux2["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(solution.x,X_test))
    m_aux2["Coeficiente de variação"].append(coefficient_of_variation(solution.x, X_test, y_test))

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:84: RuntimeWarning:

No samples with y_hat == 1 for race == 0, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:84: RuntimeWarning:

No samples with y_hat == 1 for race == 0, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalar

In [13]:
m_metrics2 = pd.DataFrame(data=m_aux2, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [14]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['MOOAcep'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = m_metrics2['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = m_metrics2['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = m_metrics2['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = m_metrics2['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

## Modelo 3) Minimax

In [15]:
other_models = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": [],
                "Modelo": [], "model_id": []}

In [16]:
import numpy as np

C_values = np.logspace(-10, 10, 150)

a_train = X_train[fair_feature].copy().astype('int')
a_val = X_val[fair_feature].copy().astype('int')

a_train[a_train==0] = -1
a_val[a_val==0] = -1


metrics_adult_min_aux = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}

idx = 0
for C in C_values:
    
    model = SKLearn_Weighted_LLR(X_train.values, y_train.values,
                 a_train.values, X_val.values,
                 y_val.values, a_val.values,
                 C_reg=C)

    mua_ini = np.ones(a_val.max() + 1)
    mua_ini /= mua_ini.sum()
    results = APSTAR(model, mua_ini, niter=200, max_patience=200, Kini=1,
                          Kmin=20, alpha=0.5, verbose=False)
    mu_best_list = results['mu_best_list']

    mu_best = mu_best_list[-1]
    model.weighted_fit(X_train.values, y_train.values, a_train.values, mu_best)
    
    other_solutions_list["Minimax"].append([idx, model])
    
    metrics_adult_min_aux["Acurácia"].append(accuracy_score(y_test, model.predict(X_test)))
    metrics_adult_min_aux["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                            X_test, y_test))
    metrics_adult_min_aux["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
    metrics_adult_min_aux["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
    
    
    other_models["Acurácia"].append(accuracy_score(y_test, model.predict(X_test)))
    other_models["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                            X_test, y_test))
    other_models["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
    other_models["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
    other_models["Modelo"].append("Minimax")
    other_models["model_id"].append(0)
    
    idx += 1

patience counter: 200 total iterations: 201
-----------------------------------------


/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0



patience counter: 200 total iterations: 201
-----------------------------------------


/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0



patience counter: 200 total iterations: 201
-----------------------------------------


...

patience counter: 3 total iterations: 201
-----------------------------------------


/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:84: RuntimeWarning:

No samples with y_hat == 1 for race == 0, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:84: RuntimeWarning:

No samples with y_hat == 1 for race == 0, returning 0



patience counter: 2 total iterations: 201
-----------------------------------------


/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:84: RuntimeWarning:

No samples with y_hat == 1 for race == 0, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

divide by zero encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:84: RuntimeWarning:

No samples with y_hat == 1 for race == 0, returning 0



patience counter: 0 total iterations: 201
-----------------------------------------
patience counter: 0 total iterations: 201
-----------------------------------------
patience counter: 0 total iterations: 201
-----------------------------------------
patience counter: 9 total iterations: 201
-----------------------------------------
patience counter: 1 total iterations: 201
-----------------------------------------
patience counter: 11 total iterations: 201
-----------------------------------------
patience counter: 24 total iterations: 201
-----------------------------------------
patience counter: 40 total iterations: 201
-----------------------------------------
patience counter: 9 total iterations: 201
-----------------------------------------
patience counter: 192 total iterations: 201
-----------------------------------------
patience counter: 193 total iterations: 201
-----------------------------------------
patience counter: 193 total iterations: 201
-------------------------

In [17]:
metrics_adult_min = pd.DataFrame(data=metrics_adult_min_aux, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [18]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['Minimax'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = metrics_adult_min['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = metrics_adult_min['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = metrics_adult_min['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = metrics_adult_min['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

## Modelo 4) Logistic regression

In [19]:
import numpy as np

C_values = np.logspace(-10, 10, 150)

metrics_adult_log_aux = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}

idx = 0
for C in C_values:
    model = LogisticRegression(C=C, max_iter=10**3, tol=10**-6)
    model.fit(X_train, y_train)
    
    other_solutions_list["LogReg"].append([idx, model])
    
    metrics_adult_log_aux["Acurácia"].append(model.score(X_test, y_test))
    metrics_adult_log_aux["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                            X_test, y_test))
    metrics_adult_log_aux["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
    metrics_adult_log_aux["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
    
    other_models["Acurácia"].append(accuracy_score(y_test, model.predict(X_test)))
    other_models["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                            X_test, y_test))
    other_models["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
    other_models["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
    other_models["Modelo"].append("LogReg")
    other_models["model_id"].append(1)
    
    idx += 1

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars



In [20]:
metrics_adult_log = pd.DataFrame(data=metrics_adult_log_aux, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [21]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['LogReg'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = metrics_adult_log['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = metrics_adult_log['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = metrics_adult_log['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = metrics_adult_log['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

## Modelo 5) Reweig

In [22]:
import numpy as np

C_values = np.logspace(-10, 10, 150)

sample_weight = calc_reweight(X_train, y_train, fair_feature)

metrics_adult_rew_aux = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}

idx = 0
for C in C_values:
    model = LogisticRegression(C=C, max_iter=10**3, tol=10**-6)
    model.fit(X_train, y_train,sample_weight=sample_weight)
    
    other_solutions_list["Reweight"].append([idx, model])
    
    metrics_adult_rew_aux["Acurácia"].append(model.score(X_test, y_test))
    metrics_adult_rew_aux["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                            X_test, y_test))
    metrics_adult_rew_aux["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
    metrics_adult_rew_aux["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
    
    other_models["Acurácia"].append(accuracy_score(y_test, model.predict(X_test)))
    other_models["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                            X_test, y_test))
    other_models["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
    other_models["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
    other_models["Modelo"].append("Reweight")
    other_models["model_id"].append(2)
    
    idx += 1

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars



In [23]:
metrics_adult_rew = pd.DataFrame(data=metrics_adult_rew_aux, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [24]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['Reweight'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = metrics_adult_rew['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = metrics_adult_rew['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = metrics_adult_rew['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = metrics_adult_rew['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

# Modelo 6) Paridade Demográfica

In [25]:
import numpy as np

C_values = np.logspace(-5, 5, 12)
c_values = np.logspace(-5, 5, 12)

metrics_adult_pp_aux = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}


idx = 0
for C in C_values:
    for c in c_values:
        try:
            model = DemographicParityClassifier(sensitive_cols=fair_feature,
                                            covariance_threshold=c, C=C, max_iter=10**3)
            model.fit(X_train, y_train)
        except:
            continue
            
        other_solutions_list["DemPar"].append([idx, model])
        
        metrics_adult_pp_aux["Acurácia"].append(model.score(X_test, y_test))
        metrics_adult_pp_aux["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                                X_test, y_test))
        metrics_adult_pp_aux["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
        metrics_adult_pp_aux["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
        
        other_models["Acurácia"].append(accuracy_score(y_test, model.predict(X_test)))
        other_models["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                                X_test, y_test))
        other_models["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
        other_models["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
        other_models["Modelo"].append("ParDem")
        other_models["model_id"].append(3)
        
        idx += 1

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars



In [26]:
metrics_adult_pp = pd.DataFrame(data=metrics_adult_pp_aux, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [27]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['DemPar'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = metrics_adult_pp['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = metrics_adult_pp['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = metrics_adult_pp['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = metrics_adult_pp['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

## Modelo 7) Equal Oportunity

In [28]:
import numpy as np

C_values = np.logspace(-5, 5, 12)
c_values = np.logspace(-5, 5, 12)

metrics_adult_eo_aux = {"Acurácia": [], "Igualdade de Oportunidade": [], "P porcento": [], "Coeficiente de variação": []}


idx = 0
for C in C_values:
    for c in c_values:
        model = EqualOpportunityClassifier(sensitive_cols=fair_feature, positive_target=True,
                                           covariance_threshold=c, C=C, max_iter=10**3)
        try:
            model.fit(X_train, y_train)
        except:
            continue
            
        other_solutions_list["EqOp"].append([idx, model])
            
        metrics_adult_eo_aux["Acurácia"].append(model.score(X_test, y_test))
        metrics_adult_eo_aux["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                                X_test, y_test))
        metrics_adult_eo_aux["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
        metrics_adult_eo_aux["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
        
        other_models["Acurácia"].append(accuracy_score(y_test, model.predict(X_test)))
        other_models["Igualdade de Oportunidade"].append(equal_opportunity_score(sensitive_column=fair_feature)(model,
                                                                                                                X_test, y_test))
        other_models["P porcento"].append(p_percent_score(sensitive_column=fair_feature)(model,X_test))
        other_models["Coeficiente de variação"].append(coefficient_of_variation(model, X_test, y_test))
        other_models["Modelo"].append("EqOp")
        other_models["model_id"].append(4)
        
        idx += 1

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:77: RuntimeWarning:

No samples with y_hat == 1 for race == 1, returning 0

/home/viguardieiro/.local/lib/python3.8/site-packages/sklego/metrics.py:156: RuntimeWarning:

invalid value encountered in double_scalars



In [29]:
metrics_adult_eo = pd.DataFrame(data=metrics_adult_eo_aux, columns=['Acurácia','Igualdade de Oportunidade','P porcento',
                                     'Coeficiente de variação'])

In [30]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['EqOp'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [0,1],
                label = 'Acc', values = metrics_adult_eo['Acurácia']),
            dict(range = [0,1],
                label = 'EO', values = metrics_adult_eo['Igualdade de Oportunidade']),
            dict(range = [0,1],
                label = 'DP', values = metrics_adult_eo['P porcento']),
            dict(range = [2,0],
                label = 'CV', values = metrics_adult_eo['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0, 1],
  …

## Visualização - Modelos Comparados

In [31]:
acc_min = min(min(other_models['Acurácia']), m_metrics['Acurácia'].min(), m_metrics2['Acurácia'].min())
acc_max = max(max(other_models['Acurácia']), m_metrics['Acurácia'].max(), m_metrics2['Acurácia'].max())
acc_min = math.floor(acc_min*1000)/1000
acc_max = math.ceil(acc_max*1000)/1000

eo_min = min(min(other_models['Igualdade de Oportunidade']), m_metrics['Igualdade de Oportunidade'].min(),
              m_metrics2['Igualdade de Oportunidade'].min())
eo_max = max(max(other_models['Igualdade de Oportunidade']), m_metrics['Igualdade de Oportunidade'].max(),
              m_metrics2['Igualdade de Oportunidade'].max())
eo_min = math.floor(eo_min*1000)/1000
eo_max = math.ceil(eo_max*1000)/1000

dp_min = min(min(other_models['P porcento']), m_metrics['P porcento'].min(), m_metrics2['P porcento'].min())
dp_max = max(max(other_models['P porcento']), m_metrics['P porcento'].max(), m_metrics2['P porcento'].max())
dp_min = math.floor(dp_min*1000)/1000
dp_max = math.ceil(dp_max*1000)/1000

cv_max = min(min(other_models['Coeficiente de variação']), m_metrics['Coeficiente de variação'].min(),
              m_metrics2['Coeficiente de variação'].min())
cv_min = max(max(other_models['Coeficiente de variação']), m_metrics['Coeficiente de variação'].max(),
              m_metrics2['Coeficiente de variação'].max())
cv_min = math.ceil(cv_min*1000)/1000
cv_max = math.floor(cv_max*1000)/1000

In [40]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['MOOError'],
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = m_metrics['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = m_metrics['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = m_metrics['P porcento']),
            dict(range = [cv_min,cv_max],
                label = 'CV', values = m_metrics['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)
fig.update_layout(font=dict(size=26))
fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

In [41]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['MOOAcep'],
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = m_metrics2['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = m_metrics2['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = m_metrics2['P porcento']),
            dict(range = [cv_min,cv_max],
                label = 'CV', values = m_metrics2['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)
fig.update_layout(font=dict(size=26))
fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

In [42]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['Minimax'],
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = metrics_adult_min['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = metrics_adult_min['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = metrics_adult_min['P porcento']),
            dict(range = [cv_min,cv_max],
                label = 'CV', values = metrics_adult_min['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)
fig.update_layout(font=dict(size=26))
fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

In [43]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=line_color['LogReg'],
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = metrics_adult_log['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = metrics_adult_log['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = metrics_adult_log['P porcento']),
            dict(range = [cv_min,cv_max],
                label = 'CV', values = metrics_adult_log['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)
fig.update_layout(font=dict(size=26))
fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

In [44]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['Reweight'],
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = metrics_adult_rew['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = metrics_adult_rew['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = metrics_adult_rew['P porcento']),
            dict(range = [cv_min,cv_max],
                label = 'CV', values = metrics_adult_rew['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)
fig.update_layout(font=dict(size=26))
fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

In [45]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['DemPar'],
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = metrics_adult_pp['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = metrics_adult_pp['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = metrics_adult_pp['P porcento']),
            dict(range = [cv_min, cv_max],
                label = 'CV', values = metrics_adult_pp['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

In [46]:
fig = go.FigureWidget(data=
    go.Parcoords(
        line_color=hist_color['EqOp'],
        #line = dict(color = new_results['dominated'], 
        #                colorscale = ['rgba(11,66,255)', 'rgba(11,0,255)']),
        dimensions = list([
            dict(range = [acc_min,acc_max],
                label = 'Acc', values = metrics_adult_eo['Acurácia']),
            dict(range = [eo_min,eo_max],
                label = 'EO', values = metrics_adult_eo['Igualdade de Oportunidade']),
            dict(range = [dp_min,dp_max],
                label = 'DP', values = metrics_adult_eo['P porcento']),
            dict(range = [cv_min, cv_max],
                label = 'CV', values = metrics_adult_eo['Coeficiente de variação']),
        ])
    ), layout = dict(height=400, width=600)#title = "Resultado para cada modelo - German")
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=26
    )
)

fig

FigureWidget({
    'data': [{'dimensions': [{'label': 'Acc',
                              'range': [0.242, 0.…

Geramos uma visualização com todos os modelos comparados (todos menos os propostos), para poupar espaço nas visualizações que serão colocadas no paper.

In [47]:
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color = other_models['model_id'], 
                    colorscale = [(0.0, hist_color['Minimax']),(0.2, hist_color['Minimax']),
                        (0.2, hist_color['LogReg']),(0.4, hist_color['LogReg']),
                        (0.4, hist_color['Reweight']),(0.6, hist_color['Reweight']),
                        (0.6, hist_color['DemPar']),(0.8, hist_color['DemPar']),
                        (0.8, hist_color['EqOp']),(1.0, hist_color['EqOp'])], 
                    showscale = False, cmin = 0, cmax = 4,
                   colorbar = {'tickvals': other_models['model_id'],
                          'ticktext': other_models['Modelo'],
                              'title': 'Approach'}),
        dimensions = list([
            dict(range = [acc_min, acc_max],
                label = 'Acc', values = other_models['Acurácia']),
            dict(range = [eo_min, eo_max],
                label = 'EO', values = other_models['Igualdade de Oportunidade']),
            dict(range = [dp_min, dp_max],
                label = 'DP', values = other_models['P porcento']),
            dict(range = [cv_min, cv_max],
                label = 'CV', values = other_models['Coeficiente de variação']),
        ]), domain = {'x': (0, 3/4)}
    ), layout = dict(height=400, width=600)
)


fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white',
    font=dict(
        size=24
    )
)

fig.show()